In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import psycopg2
from sqlalchemy import create_engine, text, Table, MetaData

from google.colab import userdata
# userdata.get('openAI_key')

Connect to Timescale DB

In [ ]:
conn_string = userdata.get('connection_string')
engine = create_engine(conn_string)

In [ ]:
drop_queries = [
    "DROP TABLE IF EXISTS Schedule;",
    "DROP TABLE IF EXISTS Student;",
    "DROP TABLE IF EXISTS Teacher;",
    "DROP TABLE IF EXISTS Course;",
    "DROP TABLE IF EXISTS Person;"
]

with engine.connect() as conn:
  for drop_query in drop_queries:
    conn.execute(text(drop_query))
  conn.commit()

Create Tables

In [ ]:
create_table_sql = """
    CREATE TABLE Person (
        id SERIAL PRIMARY KEY,
        name VARCHAR(30) NOT NULL,
        phone_number VARCHAR(15)
    );

    CREATE TABLE Teacher (
        id INTEGER PRIMARY KEY,
        rate NUMERIC(4, 2) NOT NULL,
        FOREIGN KEY (id) REFERENCES Person(id) ON DELETE CASCADE
    );

    CREATE TABLE Student (
        id INTEGER PRIMARY KEY,
        skill_level INTEGER DEFAULT 1,
        FOREIGN KEY (id) REFERENCES Person(id) ON DELETE CASCADE
    );

    CREATE TABLE Course (
        courseID SERIAL PRIMARY KEY,
        courseName VARCHAR(100) NOT NULL,
        skillLevel VARCHAR(50)
    );

    CREATE TABLE Schedule (
        studentID INTEGER,
        teacherID INTEGER,
        courseID INTEGER,
        MeetingTime TIME,
        MeetingDay VARCHAR(10),
        PRIMARY KEY (studentID, teacherID, courseID),
        FOREIGN KEY (studentID) REFERENCES Student(id) ON DELETE CASCADE,
        FOREIGN KEY (teacherID) REFERENCES Teacher(id) ON DELETE CASCADE,
        FOREIGN KEY (courseID) REFERENCES Course(courseID) ON DELETE CASCADE
    );


  """

In [ ]:
with engine.connect() as conn:
  conn.execute(text(create_table_sql))
  conn.commit()

Insert Data

In [ ]:
inserts = [
    'insert into person (id, name, phone_number)'
]

In [ ]:
person_data = [
    (1, 'Alice Johnson', '555-1234'),   # Teacher
    (2, 'Bob Smith', '555-5678'),       # Teacher
    (3, 'Charlie Davis', '555-8765'),   # Teacher
    (4, 'Dana White', '555-4321'),      # Teacher
    (5, 'Evelyn Carter', '555-2468'),    # Teacher
    (6, 'Frank Thompson', '555-1357'),   # Teacher
    (7, 'Grace Lee', '555-9876'),        # Teacher
    (8, 'Hank Miller', '555-6543'),      # Teacher
    (9, 'Ivy Nguyen', '555-7890'),       # Teacher
    (10, 'Jack Brown', '555-3210'),      # Teacher
    (11, 'Lily Adams', '555-4567'),      # Student
    (12, 'Mark Wilson', '555-7654'),     # Student
    (13, 'Nina Patel', '555-2345'),       # Student
    (14, 'Oscar Lee', '555-8760'),        # Student
    (15, 'Paula Kim', '555-1359'),        # Student
    (16, 'Quinn Brooks', '555-2589'),     # Student
    (17, 'Ryan Scott', '555-8521'),       # Student
    (18, 'Sophie Chen', '555-9630'),      # Student
    (19, 'Tina Rogers', '555-1478'),      # Student
    (20, 'Victor Gray', '555-3698')       # Student
]



teacher_data = [
    (1, 35.00),  # Alice Johnson
    (2, 40.00),  # Bob Smith
    (3, 42.50),  # Charlie Davis
    (4, 38.75),  # Dana White
    (5, 45.00),  # Evelyn Carter
    (6, 50.00),  # Frank Thompson
    (7, 37.50),  # Grace Lee
    (8, 39.25),  # Hank Miller
    (9, 41.00),  # Ivy Nguyen
    (10, 36.80)  # Jack Brown
]


student_data = [
    (11, 3),  # Lily Adams
    (12, 2),  # Mark Wilson
    (13, 1),  # Nina Patel
    (14, 3),  # Oscar Lee
    (15, 2),  # Paula Kim
    (16, 1),  # Quinn Brooks
    (17, 3),  # Ryan Scott
    (18, 3),  # Sophie Chen
    (19, 2),  # Tina Rogers
    (20, 1)   # Victor Gray
]


course_data = [
    (1, 'Classical', 1),
    (2, 'Classical', 2),
    (3, 'Classical', 3),
    (4, 'Jazz', 1),
    (5, 'Rock', 3),
    (6, 'Pop', 3),
    (7, 'Blues', 3),
    (8, 'Hip Hop', 2),
    (9, 'Folk', 1),
    (10, 'Country', 2),
    (11, 'Electronic', 3),
    (12, 'Reggae', 1)
]

schedule_data = [
    (11, 4, 1, '10:00:00', 'Monday'),
    (12, 8, 2, '14:00:00', 'Wednesday'),
    (12, 3, 1, '09:00:00', 'Tuesday'),
    (12, 1, 2, '13:00:00', 'Thursday'),
    (15, 2, 3, '11:00:00', 'Friday'),
    (13, 2, 4, '15:00:00', 'Saturday'),
    (13, 2, 5, '12:00:00', 'Monday'),
    (20, 5, 6, '16:00:00', 'Tuesday'),
    (19, 10, 7, '08:00:00', 'Wednesday'),
    (20, 9, 8, '18:00:00', 'Thursday')
]

# Define the SQL queries for each table
sql_person_insert = """
INSERT INTO Person (id, name, phone_number) VALUES (%s, %s, %s)
"""

sql_teacher_insert = """
INSERT INTO Teacher (id, rate) VALUES (%s, %s)
"""

sql_student_insert = """
INSERT INTO Student (id, skill_level) VALUES (%s, %s)
"""

sql_course_insert = """
INSERT INTO Course (courseID, courseName, skillLevel) VALUES (%s, %s, %s)
"""

sql_schedule_insert = """
INSERT INTO Schedule (studentID, teacherID, courseID, MeetingTime, MeetingDay) VALUES (%s, %s, %s, %s, %s)
"""

# Connect to the database and insert the data
with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()

    # Insert data into the Person table
    cursor.executemany(sql_person_insert, person_data)

    # Insert data into the Teacher table
    cursor.executemany(sql_teacher_insert, teacher_data)

    # Insert data into the Student table
    cursor.executemany(sql_student_insert, student_data)

    # Insert data into the Course table
    cursor.executemany(sql_course_insert, course_data)

    # Insert data into the Schedule table
    cursor.executemany(sql_schedule_insert, schedule_data)

    # Commit the transaction
    conn.commit()

In [ ]:
def show_table(table_name):
    select = f"select * from {table_name}"
    with psycopg2.connect(conn_string) as conn:
        cursor = conn.cursor()
        cursor.execute(select)
        results = cursor.fetchall()
        for row in results:
            print(row)

In [ ]:
show_table('person')

(1, 'Alice Johnson', '555-1234')
(2, 'Bob Smith', '555-5678')
(3, 'Charlie Davis', '555-8765')
(4, 'Dana White', '555-4321')
(5, 'Evelyn Carter', '555-2468')
(6, 'Frank Thompson', '555-1357')
(7, 'Grace Lee', '555-9876')
(8, 'Hank Miller', '555-6543')
(9, 'Ivy Nguyen', '555-7890')
(10, 'Jack Brown', '555-3210')
(11, 'Lily Adams', '555-4567')
(12, 'Mark Wilson', '555-7654')
(13, 'Nina Patel', '555-2345')
(14, 'Oscar Lee', '555-8760')
(15, 'Paula Kim', '555-1359')
(16, 'Quinn Brooks', '555-2589')
(17, 'Ryan Scott', '555-8521')
(18, 'Sophie Chen', '555-9630')
(19, 'Tina Rogers', '555-1478')
(20, 'Victor Gray', '555-3698')


OpenAI Usage

In [ ]:
import json
from openai import OpenAI
import os
import sqlite3
from time import time

def runSql(query):
  with psycopg2.connect(conn_string) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()


# OPENAI
openAiClient = OpenAI(
    api_key = userdata.get("openaiKey"),
    organization = userdata.get("orgId")
)

def getChatGptResponse(content):
    stream = openAiClient.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": content}],
        stream=True,
    )

    responseList = []
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            responseList.append(chunk.choices[0].delta.content)

    result = "".join(responseList)
    return result


# strategies
commonSqlOnlyRequest = " Give me a postgre select statement that answers the question. Only respond with postgre syntax. If there is an error do not expalin it!"
strategies = {
    "zero_shot": create_table_sql + commonSqlOnlyRequest,
    "single_domain_double_shot": (
        create_table_sql +
        " Which students are not enrolled in any courses? " +
        "\nSELECT s.id, p.name " +
        "\nFROM student s " +
        "\nJOIN person p ON s.id = p.id " +
        "\nLEFT JOIN schedule sch ON s.id = sch.studentID " +
        "\nWHERE sch.courseID IS NULL;" +
        "\n" + commonSqlOnlyRequest
    )
}

questions = [
    "Which teachers have the most students?",
    "Which students are in multiple classes",
    "Which teacher has the highest rate",
    "How many students are enrolled in each course?",
    "Which courses have no students scheduled?",
    "What is the average rate of teachers?",
    "List all teachers and the courses that they are teaching",
    "What are the names of courses offered for the skill level 1?",
    "What is the average skill level of the students?"
    # "I need insert sql into my tables can you provide good unique data?"
]

def sanitizeForJustSql(value):
    gptStartSqlMarker = "```sql"
    gptEndSqlMarker = "```"
    if gptStartSqlMarker in value:
        value = value.split(gptStartSqlMarker)[1]
    if gptEndSqlMarker in value:
        value = value.split(gptEndSqlMarker)[0]

    return value

for strategy in strategies:
    responses = {"strategy": strategy, "prompt_prefix": strategies[strategy]}
    questionResults = []
    for question in questions:
        print(question)
        error = "None"
        try:
            sqlSyntaxResponse = getChatGptResponse(strategies[strategy] + " " + question)
            sqlSyntaxResponse = sanitizeForJustSql(sqlSyntaxResponse)
            print(sqlSyntaxResponse)
            queryRawResponse = str(runSql(sqlSyntaxResponse))
            print(queryRawResponse)
            friendlyResultsPrompt = "I asked a question \"" + question +"\" and the response was \""+queryRawResponse+"\" Please, just give a concise response in a more friendly way? Please do not give any other suggests or chatter."
            friendlyResponse = getChatGptResponse(friendlyResultsPrompt)
            print(friendlyResponse)
        except Exception as err:
            error = str(err)
            print(err)

        questionResults.append({
            "question": question,
            "sql": sqlSyntaxResponse,
            "queryRawResponse": queryRawResponse,
            "friendlyResponse": friendlyResponse,
            "error": error
        })

    responses["questionResults"] = questionResults

Which teachers have the most students?

SELECT t.id, p.name, COUNT(s.id) AS student_count
FROM Teacher t
JOIN Schedule sch ON t.id = sch.teacherID
JOIN Student s ON sch.studentID = s.id
JOIN Person p ON t.id = p.id
GROUP BY t.id, p.name
ORDER BY student_count DESC;

[(2, 'Bob Smith', 3), (8, 'Hank Miller', 1), (3, 'Charlie Davis', 1), (10, 'Jack Brown', 1), (9, 'Ivy Nguyen', 1), (5, 'Evelyn Carter', 1), (1, 'Alice Johnson', 1), (4, 'Dana White', 1)]
Sure! The teacher with the most students is Bob Smith with 3 students.
Which students are in multiple classes

SELECT s.id, p.name
FROM Student s
JOIN Person p ON s.id = p.id
JOIN Schedule sch ON s.id = sch.studentID
GROUP BY s.id, p.name
HAVING COUNT(sch.courseID) > 1;

[(20, 'Victor Gray'), (12, 'Mark Wilson'), (13, 'Nina Patel')]
The students in multiple classes are Victor Gray, Mark Wilson, and Nina Patel.
Which teacher has the highest rate

SELECT id, rate FROM Teacher ORDER BY rate DESC LIMIT 1;

[(6, Decimal('50.00'))]
The teacher wi